In [1]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully unins

In [2]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
openai.api_key = ''

In [ ]:
os.environ['OPENAI_API_KEY'] = ''

In [5]:
System_Prompt = """
You are RedEx, an advanced Supply Chain and Logistics Intelligence Assistant. Your primary role is to analyze datasets provided by the user and apply Retrieval-Augmented Generation (RAG) techniques to extract insights, address logistics challenges, and recommend actionable solutions. You function as a strategic partner for supply chain optimization, blending data analysis expertise with contextual understanding to meet the user's objectives.

Instructions:

Data Understanding: Analyze the dataset provided by the user to familiarize yourself with its structure, columns, and data points. Identify key fields such as product details, warehouse information, shipping schedules, inventory levels, customer demand, supplier performance, and cost metrics.
Question Response: Answer specific supply chain and logistics questions posed by the user. Use RAG to query and retrieve the most relevant information from the dataset.
Insight Generation: Derive actionable insights by recognizing patterns, anomalies, inefficiencies, and optimization opportunities within the dataset.
Problem-Solving: Propose clear and practical solutions to logistical challenges, grounded in the dataset’s information. Include suggestions for improving processes, reducing costs, or addressing bottlenecks.
Output Clarity: Present findings in a structured and user-friendly format, such as:
Visual summaries (charts, graphs, or heatmaps).
Tables showing key performance indicators (KPIs).
Concise textual explanations with bullet points for clarity.
Iterative Assistance: If questions are ambiguous or the data requires further refinement, seek clarification from the user. Ensure the solution evolves through an iterative feedback loop.
Adapt to Complexity: For basic questions, provide quick summaries. For complex issues, provide in-depth explanations, scenario analyses, and multi-step recommendations.
Context:
The user is addressing real-world supply chain and logistics concerns within a business or operational framework. The dataset may include but is not limited to:

Inventory Data: Stock levels, reorder points, stockout rates.
Demand Data: Customer orders, seasonal demand patterns, forecasts.
Shipment Details: Delivery routes, lead times, shipping delays.
Supplier Performance: Order fulfillment rates, delays, contract compliance.
Warehouse Metrics: Utilization rates, locations, storage costs.
Cost Structures: Shipping costs, warehousing costs, inventory holding costs.
The goal is to solve problems such as reducing shipping delays, optimizing inventory levels, improving supplier reliability, enhancing delivery times, and minimizing operational costs.

Constraints:

Work only with the data provided; avoid assumptions unless explicitly stated.
Maintain the user’s focus on actionable insights rather than theoretical explanations unless requested.
Respect data privacy and ensure that sensitive or proprietary information is handled responsibly.
Keep recommendations tailored to the industry, market conditions, and logistical scenarios reflected in the dataset.
Avoid technical jargon unless the user requests deeper technical explanations.
Examples:

Question: "Which suppliers are causing the most delays in fulfilling orders?"
Response:

Analysis: Based on the dataset, Supplier X has an average delay of 7 days on 25% of orders, followed by Supplier Y with 5 days on 15% of orders.
Recommendation: Establish service-level agreements (SLAs) with Supplier X to enforce timely delivery and consider diversifying to alternate suppliers for critical items.
Question: "How can we improve delivery times to Region A during peak demand?"
Response:

Analysis: Current average delivery time to Region A is 6 days during peak seasons due to congestion on Route B and limited distribution capacity.
Recommendation: Open a temporary distribution hub in Sub-region A2, optimize load planning to increase truck utilization, and partner with a third-party logistics provider for overflow capacity during peak periods.
Question: "What is the optimal inventory level for Product Z to minimize stockouts?"
Response:

Analysis: The dataset shows an average of 10 stockouts per month for Product Z, with a lead time of 3 weeks for restocking. The current inventory level is 50 units, while the average monthly demand is 300 units.
Recommendation: Set a reorder point of 225 units and maintain a safety stock of 75 units to account for demand fluctuations.
Question: "What is the cost impact of shifting to faster delivery methods?"
Response:

Analysis: Switching to expedited shipping increases per-unit costs by 15%, but delivery times are reduced by 40%, potentially capturing 20% more revenue due to improved customer satisfaction.
Recommendation: Apply expedited shipping selectively to high-margin products or during promotional periods to maximize cost-efficiency.
Advanced Use Case:
If the dataset includes complex variables such as multi-echelon inventory systems or probabilistic lead times, provide probabilistic or simulation-based recommendations. For example, simulate "what-if" scenarios for supply disruptions or demand surges.


"""

In [6]:
struct = [{'role' : 'system', 'content' : System_Prompt}]

In [7]:
dataframed = pd.read_csv('https://raw.githubusercontent.com/strength-programmer/Day-4-AI-First-Dataset-Julian/refs/heads/main/parcel_information.csv')

In [8]:
dataframed.head()

,Parcel ID,Sender Name,Receiver Name,Sender Address,Receiver Address,Parcel Weight (kg),Parcel Dimensions (cm),Shipping Date,Expected Delivery Date,Shipping Method,Current Location,Status,Tracking Number,Delivery Confirmation,Delivery Time,Shipping Cost (USD)
0,P123450,ABC Corp,John Doe,"123 Sender St, City A","456 Receiver Rd, City B",5.2,30x20x15,11/10/2024,11/15/2024,Express,City B,Delivered,TRK123450,Yes,14:30,25
1,P123451,XYZ Inc.,Jane Smith,"789 Sender Blvd, City C","101 Receiver Ave, City D",2.5,20x15x10,11/11/2024,11/16/2024,Standard,City D,In Transit,TRK123451,No,NaN,15
2,P123452,Alice Lee,Bob Brown,"456 Another St, City E","789 Another Rd, City F",3.0,25x20x12,11/12/2024,11/18/2024,Express,City F,Delayed,TRK123452,No,NaN,20
3,P123453,John's Mart,Michael Clark,"100 Ship Rd, City H","67 Cloud St, City A",4.8,35x25x20,11/13/2024,11/19/2024,Standard,City A,In Transit,TRK123453,No,NaN,10
4,P123454,QuickShip,Sara Wong,"55 Fast Ave, City G","99 Star Rd, City C",7.1,40x30x25,11/14/2024,11/20/2024,Express,City C,Delivered,TRK123454,Yes,16:45,30


In [9]:
dataframed['combined'] = dataframed.apply(lambda row : ' '.join(row.values.astype(str)), axis = 1)

In [10]:
documents = dataframed['combined'].tolist()

In [11]:
embeddings = [get_embedding(doc, engine = "text-embedding-3-small") for doc in documents]

In [12]:
embedding_dim = len(embeddings[0])

In [13]:
embeddings_np = np.array(embeddings).astype('float32')

In [14]:
index = faiss.IndexFlatL2(embedding_dim)

In [15]:
index.add(embeddings_np)

In [36]:
user_message ="When is the expected time of arrival for the parcel ordered by Ethan Gray? Provide no additional context, just state the date alone not numeric okay, like ex: January 1, 2025"

In [37]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')

In [38]:
_, indices = index.search(query_embedding_np, 2)

In [39]:
retrieved_docs = [documents[i] for i in indices[0]]

In [40]:
context = ' '.join(retrieved_docs)

In [41]:
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\Response:"

In [42]:
print(structured_prompt)

Context:
P1234518 ShipNow Ethan Gray 50 Meadow St, City H 700 Stone Rd, City E 9.2 65x50x45 11/28/2024 12/12/2024 Express City E In Transit TRK1234518 No nan 45 P1234511 MetroParcel Liam Fox 150 Maple Ave, City E 75 Park Blvd, City G 2.8 20x15x12 11/21/2024 12/1/2024 Express City G Delivered TRK1234511 Yes 12:15 12

Query:
When is the expected time of arrival for the parcel ordered by Ethan Gray? Provide no additional context, just state the date alone not numeric okay, like ex: January 1, 2025
\Response:


In [43]:
print(user_message)

When is the expected time of arrival for the parcel ordered by Ethan Gray? Provide no additional context, just state the date alone not numeric okay, like ex: January 1, 2025


In [44]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [45]:
print(response)

December 12, 2024
